In [5]:
#Alternative : 
#import files
#from local_navigation import prox
#import class
#from thymioV2 import thymio_robot
#import librairies
import time
import numpy as np
import math 
import cv2


In [6]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()

ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

In [8]:
from src.Motion_Control import thymio as th
from src.Global_Nav import helpers_global as gb
from src.Vision import vision as vs
#from src.Vision import vision_test as vs2



In [9]:
await th.move_to_goal(client, [2,4, 3*np.pi/4], [4,5], 100)


-1.5707963267948966


In [8]:
await th.move_forward(client, 100, 2)

In [19]:
await th.stop_motor(node )

In [7]:
x =[3,4]
y = [5]
w = np.append(x, y)
w

array([3, 4, 5])

In [8]:
MAP_SHAPE = (1000,700)
REFRAME = True 

In [5]:
MAP_SHAPE = (1000,700)
REFRAME = True 
cap = cv2.VideoCapture(0)

ret,frame = cap.read()
path = vs.get_grid_fixed_map(frame,(35,50),50)
while True:
    ret,frame = cap.read()
    Tmap = vs.get_warp(cap,MAP_SHAPE,10,1)
    if ret:
        # maps capture to map
        if REFRAME:
            frame = cv2.warpPerspective(frame,Tmap,MAP_SHAPE)
        # maps BGR to HLS color space for simplicity
        HLS = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS_FULL)

        bool_pos ,center,scale,orient = vs.get_Robot_position_orientation(HLS, [5,5])
        center_idx = th.convert_to_idx(center, 2)
        if gb.test_if_goal(center, path[-1]):
             break

        check_point = gb.next_checkpoint(path, center_idx)
        await th.move_to_goal(client, np.append(center, orient), check_point, 100)



    

NameError: name 'MAP_SHAPE' is not defined

In [11]:
#To unlock the Robot
await node.unlock()

{'error_code': 2}

In [ ]:
#Thymio control functions 
def motors(l_speed=500, r_speed=500):
    return {
        "motor.left.target": [l_speed],
        "motor.right.target": [r_speed],
    }

async def forward(node, motor_speed):
    await node.set_variables(motors(motor_speed,motor_speed))

async def motorset(node,motor_speed_left,motor_speed_right):
    await node.set_variables(motors(motor_speed_left,motor_speed_right))

async def rotate(node,theta, motor_speed): #theta is in radians
    direction_rot=(theta>=0)-(theta<0)
    await node.set_variables(motors(motor_speed*direction_rot, -motor_speed*direction_rot))
    # wait time to get theta 1.44 is the factor to correct
    time=(theta)*100/motor_speed*1.44
    await(client.sleep(time))
    # stop the robot
    await node.set_variables(motors(0, 0))

async def stop_motor(node):
    await node.set_variables(motors(0,0))

async def get_proximity_values(node):
    # Wait for the Thymio node
    node = await client.wait_for_node()
    # Wait for the proximity sensor variables
    await node.wait_for_variables({"prox.horizontal"})
    # Get the proximity values : v: Stands for "variables" and is used to access the cached variable values.
    proximity_values = node.v.prox.horizontal
    # Return the value of the front proximity sensor (index 2)
    return proximity_values[0:5]

# Run the asynchronous function and print the result
proximity_values = await get_proximity_values(node)
print(f"Front Proximity Sensor Value: {proximity_values}")
#await rotate(np.pi,100)


In [ ]:
#Navigation control
#find_index if robot abs coord is close enough to a position of the A*paths
def find_index_path_if_close(path_coord, position_robot, epsilon):
    # Displacement vector
    vect_disp = path_coord - position_robot

    # Calculate the distance between the robot's position and the path
    dist = np.sqrt(vect_disp[:, 0]**2 + vect_disp[:, 1]**2)
    
    # Find indices where the distance is less than epsilon
    indices = np.where(dist < epsilon)[0]
    
    if indices.size == 0:
        # No indices found
        #print("No indices found where the distance is less than epsilon.")
        return None
    else:
        # Return the first index
        #print(indices)
        return indices[0]

In [ ]:
await stop_motor()

In [ ]:
#local navigation

state=1
async def local_navigation2():
    proximity_values = await get_proximity_values()
    global state

    w_l = [40,  20, -20, -20, -40,  30, -10, 8, 0]
    
    w_r = [-40, -20, -20,  20,  40, -10, 30, 0, 8]

    # Scale factors for sensors and constant factor
    sensor_scale = 2000
    
    x = [0,0,0,0,0,0,0,0,0]
    
    if state != 0:
        for i in range(5):
            # Get and scale inputs
            x[i] = proximity_values[i] // sensor_scale
        
        y = [100,100]    
        
        for i in range(len(x)):    
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + x[i] * w_l[i]
            y[1] = y[1] + x[i] * w_r[i]
    else: 
        # In case we would like to stop the robot
        y = [0,0] 
    
    # Set motor powers
    await motorset(y[0],y[1])

In [ ]:
await stop_motor()

In [ ]:
#MAIN CODE 1: 

#parameters
path_step=0 #This is the index of where we are in the path.
path_coord = np.array([[1,2], [2,3], [3,4]])
b = np.array([3.2, 4.5])
epsilon = 0.5
local_obstacle=False
threshold_obst=3500


#to implement in a different way thanks to filtering : 
robot_position=1
end_position=2

async def main():
    global local_obstacle
    
    while(robot_position!=end_position):
        sens = await get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True
            start_time = time.time()

        if(local_obstacle):
            sens = await get_proximity_values()
            #sens=list(node["prox.horizontal"])
            await local_navigation2()
            local_obstacle=False
        
        #if(find_index_path_if_close(path_coord,b,epsilon) and time.time-start_time>2): 
            #path_step=find_index_path_if_close(path_coord, position_robot, epsilon)
            #local_navigation=False

        if not local_obstacle: 
            node.send_set_variables(motors(50, 50))
            #Function input : absolute velocity wanted ; output wheel velocity
            #u=speed along x axis
            #v=speed along y axis
            #apply compute wheel velocity to robot in the while loop
            pass

await main()